# Etapa 1: Importar Bibliotecas Necessárias
Importar as bibliotecas é fundamental para acessar funções e ferramentas de manipulação de dados, lógica fuzzy e sistemas de controle. Sem elas, o restante do notebook não pode ser executado.

In [10]:
# Etapa 1: Importar Bibliotecas Necessárias
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Etapa 2: Carregar ou Simular o Conjunto de Dados
Carregar o conjunto de dados fornece os dados reais necessários para a análise. Se o conjunto de dados não estiver disponível, simulá-lo garante que o fluxo de trabalho possa continuar e ser testado.

In [11]:

# 2. Load or Simulate Dataset
def load_or_simulate_dataset(dataset_name, default_rows=1000):
    """Loads the specified dataset from Kaggle or simulates it if not found."""
    try:
        if dataset_name == "diabetes":
            return pd.read_csv('diabetes.csv')
        else:
            raise FileNotFoundError
    except:
        print(f"⚠️ Dataset '{dataset_name}' not found. Simulating data.")
        return simulate_diabetes_dataset(default_rows)

def simulate_diabetes_dataset(rows):
    """Creates a realistic synthetic diabetes dataset."""
    return pd.DataFrame({
        'Pregnancies': np.random.randint(0, 10, rows),
        'Glucose': np.random.normal(120, 30, rows).clip(70, 200),
        'BloodPressure': np.random.normal(70, 15, rows).clip(40, 100),
        'SkinThickness': np.random.normal(20, 5, rows),
        'Insulin': np.random.normal(100, 50, rows),
        'BMI': np.random.normal(30, 5, rows),
        'DiabetesPedigreeFunction': np.random.uniform(0.1, 1.5, rows),
        'Age': np.random.randint(20, 70, rows),
        'Outcome': np.random.binomial(1, 0.3, rows)
    })

diabetes_data = load_or_simulate_dataset("diabetes")
print("✅ Diabetes dataset loaded.")

✅ Diabetes dataset loaded.


# Etapa 3: Definir Variáveis Fuzzy
Definir as variáveis fuzzy configura os parâmetros de entrada e saída para o sistema de lógica fuzzy. Essas variáveis representam as características e resultados a serem analisados.

In [12]:
# 3. Define Fuzzy Variables
bp = ctrl.Antecedent(np.arange(40, 110, 1), 'BloodPressure')
age = ctrl.Antecedent(np.arange(20, 80, 1), 'Age')
glucose = ctrl.Antecedent(np.arange(70, 200, 1), 'Glucose')
heart_attack_risk = ctrl.Consequent(np.arange(0, 101, 1), 'HeartAttackRisk')



# Etapa 4: Criar Conjuntos Fuzzy (Funções de Pertinência)
As funções de pertinência definem como cada variável é categorizada (por exemplo, baixa, normal, alta). Isso é fundamental para o sistema fuzzy interpretar valores do mundo real.

In [13]:
# 4. Create Fuzzy Sets (Membership Functions)
bp['low'] = fuzz.trimf(bp.universe, [40, 40, 65])
bp['normal'] = fuzz.trimf(bp.universe, [60, 75, 90])
bp['high'] = fuzz.trimf(bp.universe, [85, 110, 110])

age['young'] = fuzz.trimf(age.universe, [20, 20, 40])
age['middle'] = fuzz.trimf(age.universe, [35, 50, 65])
age['senior'] = fuzz.trimf(age.universe, [60, 80, 80])

glucose['low'] = fuzz.trimf(glucose.universe, [70, 70, 100])
glucose['normal'] = fuzz.trimf(glucose.universe, [90, 120, 150])
glucose['high'] = fuzz.trimf(glucose.universe, [140, 200, 200])

heart_attack_risk['low'] = fuzz.trimf(heart_attack_risk.universe, [0, 0, 50])
heart_attack_risk['medium'] = fuzz.trimf(heart_attack_risk.universe, [25, 50, 75])
heart_attack_risk['high'] = fuzz.trimf(heart_attack_risk.universe, [50, 100, 100])

# Etapa 5: Definir Regras Fuzzy
As regras fuzzy estabelecem as relações entre as variáveis de entrada e a saída. Elas são a lógica central que direciona a tomada de decisão no sistema fuzzy.

In [14]:

# 5. Define Fuzzy Rules
rule1 = ctrl.Rule(bp['normal'] & age['young'] & glucose['normal'], heart_attack_risk['low'])
rule2 = ctrl.Rule(bp['high'] & (age['middle'] | age['senior']), heart_attack_risk['high'])
rule3 = ctrl.Rule(bp['normal'] & age['middle'] & glucose['normal'], heart_attack_risk['medium'])
rule4 = ctrl.Rule(bp['high'] | glucose['high'], heart_attack_risk['high'])

# Etapa 6: Construir e Simular o Sistema de Controle Fuzzy
Construir e simular o sistema de controle combina as regras e variáveis, permitindo que o motor de lógica fuzzy processe entradas e gere saídas.

In [15]:

# 6. Build and Simulate the Fuzzy Control System
heart_attack_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
heart_attack_simulation = ctrl.ControlSystemSimulation(heart_attack_ctrl)

# Etapa 7: Aplicar o Sistema a um Exemplo de Dados
Aplicar o sistema a um exemplo de dados demonstra como o modelo fuzzy funciona na prática, usando dados reais ou simulados.

In [20]:

# 7. Apply the System to a Sample Data Point
sample_data = diabetes_data.iloc[10]
heart_attack_simulation.input['BloodPressure'] = sample_data['BloodPressure']
heart_attack_simulation.input['Age'] = sample_data['Age']
heart_attack_simulation.input['Glucose'] = sample_data['Glucose']

# Etapa 8: Exibir o Resultado
Exibir o resultado permite interpretar a análise fuzzy, fornecendo insights sobre o risco de ataque cardíaco com base nos dados de entrada.

In [21]:



# 8. Display the Output
heart_attack_simulation.compute()
print("\n--- Fuzzy Logic Analysis for Heart Attack Risk ---")
print(f"Input Data:")
print(f"  - Blood Pressure: {sample_data['BloodPressure']:.2f} mmHg")
print(f"  - Age: {sample_data['Age']:.2f} years")
print(f"  - Glucose: {sample_data['Glucose']:.2f} mg/dL")

print(f"\nFuzzy Logic Output:")
print(f"  - Calculated Heart Attack Risk: {heart_attack_simulation.output['HeartAttackRisk']:.2f}%")


--- Fuzzy Logic Analysis for Heart Attack Risk ---
Input Data:
  - Blood Pressure: 92.00 mmHg
  - Age: 30.00 years
  - Glucose: 110.00 mg/dL

Fuzzy Logic Output:
  - Calculated Heart Attack Risk: 78.31%
